In [ ]:
import numpy as np
import scipy.stats as sps
from random import choices

In [5]:
def changementeps(distances, eps, poids, alpha, n):
    epsilons = np.sort(distances[distances < eps])[::-1]
    nbops = len(epsilons)
    ESS = 1 / np.sum(poids ** 2)
    k = 0
    nouvpoids = poids
    while k < nbops - 1:
        nouveps = epsilons[k]
        for i in range(n):
            if poids[i] != 0:
                if distances[i] > eps or distances[i] > nouveps:
                    nouvpoids[i] = 0
                else:
                    nouvpoids[i] = 1 / distances[i]
        nouvpoids = nouvpoids / np.sum(nouvpoids)
        nouvESS = 1 / np.sum(nouvpoids ** 2)
        if nouvESS < (alpha * ESS):
            return nouveps, nouvpoids, nouvESS
        k += 1
    return eps, poids, ESS

def chgtpar(simui, distancesi, sd, eps, data, dprior, dist, n, i):
    var = np.random.multivariate_normal(np.zeros(len(sd)), cov=sd)
    nsimu = simui + np.append(np.mean(var), var)
    comp = dprior(nsimu, nsimu[1])
    ndistances = dist(nsimu)
    u = np.random.uniform(1)
    if i == 0:
        print("prior")
        print(dprior(nsimu, nsimu[1]))
        print(dprior(simui, simui[1]))
    if ndistances > eps:
        return simui, distancesi
    elif distancesi > eps:
        return nsimu, ndistances
    elif u < distancesi / ndistances:
        print("C")
        return nsimu, ndistances
    else:
        return simui, distancesi

def gibbs_abc(data, thetini, hyperini, sigm, var, nbeps1, nbeps2, nbpts, qq):
    reshyper = np.zeros(nbpts + 1)
    resparam = np.zeros((len(thetini), nbpts + 1))
    reshyper[0] = hyperini
    resparam[:, 0] = thetini
    resdist = np.zeros(nbpts)

    for i in range(1, nbpts + 1):
        resdist[i - 1] = 0
        VV, dist_param = changementeps(data, reshyper[i - 1], var, sigm, nbeps1)
        resparam[:, i] = VV
        resdist[i - 1] += dist_param
        WW, dist_hyper = gibbshyper(resparam[:, i], nbeps2, var)
        reshyper[i] = WW

    return reshyper, resparam, resdist


In [7]:

gibbs_abc(data, rprior(50)[0], 50, 150, 50, 0.9, 50, 50, None)


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.

In [12]:
def changementeps(distances, eps, poids, alpha, n):
    epsilons = np.sort(distances[distances < eps])[::-1]
    nbops = len(epsilons)
    ESS = 1 / np.sum(poids ** 2)
    k = 0
    nouvpoids = poids
    while k < nbops - 1:
        nouveps = epsilons[k]
        for i in range(n):
            if poids[i] != 0:
                if distances[i] > eps or distances[i] > nouveps:
                    nouvpoids[i] = 0
                else:
                    nouvpoids[i] = 1 / distances[i]
        nouvpoids = nouvpoids / np.sum(nouvpoids)
        nouvESS = 1 / np.sum(nouvpoids ** 2)
        if nouvESS < (alpha * ESS):
            return nouveps, nouvpoids, nouvESS
        k += 1
    return eps, poids, ESS

def chgtpar(simui, distancesi, sd, eps, data, dprior, dist, n, i):
    var = np.random.multivariate_normal(np.zeros(len(sd)), cov=sd)
    nsimu = simui + np.append(np.mean(var), var)
    comp = dprior(nsimu, nsimu[1])
    ndistances = dist(nsimu)
    u = np.random.uniform(1)
    if i == 0:
        print("prior")
        print(dprior(nsimu, nsimu[1]))
        print(dprior(simui, simui[1]))
    if ndistances > eps:
        return simui, distancesi
    elif distancesi > eps:
        return nsimu, ndistances
    elif u < distancesi / ndistances:
        print("C")
        return nsimu, ndistances
    else:
        return simui, distancesi

def gibbs_abc(data, thetini, hyperini, sigm, var, nbeps1, nbeps2, nbpts, qq):
    reshyper = np.zeros(nbpts + 1)
    resparam = np.zeros((len(thetini), nbpts + 1))
    reshyper[0] = hyperini
    resparam[:, 0] = thetini
    resdist = np.zeros(nbpts)

    for i in range(1, nbpts + 1):
        resdist[i - 1] = 0
        VV, dist_param = changementeps(data, reshyper[i - 1], var, sigm, nbeps1)
        resparam[:, i] = VV
        resdist[i - 1] += dist_param
        WW, dist_hyper = gibbshyper(resparam[:, i], nbeps2, var)
        reshyper[i] = WW

    return reshyper, resparam, resdist


In [ ]:

# Appliquer l'algorithme Gibbs ABC
gibbs_abc(data, 50, 150, 50, 0.85, rprior, dprior, dist, l)
